In [271]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [272]:
df = pd.read_json('reviews_Baby_5.json', lines=True, encoding='utf-8')
display(df.head())
print(len(df))

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1HK2FQW6KXQB2,097293751X,"Amanda Johnsen ""Amanda E. Johnsen""","[0, 0]",Perfect for new parents. We were able to keep ...,5,Awesine,1373932800,"07 16, 2013"
1,A19K65VY14D13R,097293751X,angela,"[0, 0]",This book is such a life saver. It has been s...,5,Should be required for all new parents!,1372464000,"06 29, 2013"
2,A2LL1TGG90977E,097293751X,Carter,"[0, 0]",Helps me know exactly how my babies day has go...,5,Grandmother watching baby,1395187200,"03 19, 2014"
3,A5G19RYX8599E,097293751X,cfpurplerose,"[0, 0]",I bought this a few times for my older son and...,5,repeat buyer,1376697600,"08 17, 2013"
4,A2496A4EWMLQ7,097293751X,C. Jeter,"[0, 0]",I wanted an alternative to printing out daily ...,4,Great,1396310400,"04 1, 2014"


160792


데이터 개수가 많아서 2000개만 랜덤으로 추출하도록 하겠습니다. 먼저 부정적인 데이터(0-3점) 중 1000개를 추출하고, 긍정적인 데이터(4-5점) 중에서 1000개를 추출하여 총 2000개의 리뷰를 추출하도록 하겠습니다.

# 부정 / 긍정 데이터 분리

In [273]:
negative_reviews = df[ df['overall'] < 4 ]
positive_reviews = df[ df['overall'] >= 4 ]
print(len(positive_reviews))
print(len(negative_reviews))

126525
34267


# 부정 / 긍정 데이터 샘플링

In [274]:
neg_samples = negative_reviews.sample(n=1000, replace=False)
pos_samples = positive_reviews.sample(n=1000, replace=False)
print(len(neg_samples))
print(len(pos_samples))

1000
1000


# 리뷰 텍스트만 가져오기

In [275]:
neg_list = neg_samples['reviewText'].values.tolist()
pos_list = pos_samples['reviewText'].values.tolist()
display(neg_list[0])
display(pos_list[0])

"I registered for this swing for my son as it was one of the few non-feminine swings I could find.  I thought it was very cute & gender neutral.  I have been extremely disappointed.  The motor on it is a joke and what's worse, we exchanged it & had the same problems so I know it isn't just one bad swing.  The motor is loud enough that we thought it was broken when we first tried it out, but what's worse is it barely has the power to swing my 12 pound son.  Literally, if I leave it to swing on its own, it swings ONE INCH either way with my son in it & he is only half the recommended weight capacity.  Even when he was newly born (8 lb's), it barely swung him.  So I sit by the swing & push it manually.  Also, the recline is not flat enough for a newborn because my son's head rolls forward & he slumps in the chair no matter what I do.  Not only would I not buy this swing again, but after all the product recalls I've seen in the news, I'd be reluctant to purchase anything Fisher-Price in th

"My baby absolutely loves these. She seems to prefer the soft, flexible type of teether. I just bought a second pack of them. They're easy to sanitize as well (I steam them in the microwave steam bag I have for my medela pumping stuff) and the shape is both easy for her to grab as well as easy to clip to things with baby links."

# Counter 객체 만들어서 각 단어 빈도수 저장

## 긍정 리뷰 단어 빈도수 저장

In [276]:
# 각 리뷰를 word 별로 split한다
temp = []
temp.extend([sentence.split(' ') for sentence in pos_list])
# split된 각 리뷰를 flatten하게 만든다
pos_words = []
for sublist in temp:
    for item in sublist:
        pos_words.append(item)
# flatten 한 words 배열을 Counter 객체에 전달하여 단어별 빈도수를 저장한다
pos_counter = Counter(pos_words)
sorted(pos_counter.items(), reverse=True, key=lambda el: el[1])

[('the', 4143),
 ('and', 3029),
 ('to', 2801),
 ('I', 2473),
 ('a', 2256),
 ('it', 1732),
 ('', 1671),
 ('is', 1623),
 ('for', 1399),
 ('of', 1170),
 ('in', 1130),
 ('this', 997),
 ('that', 929),
 ('my', 877),
 ('with', 861),
 ('on', 807),
 ('have', 756),
 ('but', 695),
 ('so', 664),
 ('are', 641),
 ('was', 620),
 ('The', 521),
 ('you', 491),
 ('as', 471),
 ('not', 461),
 ('we', 437),
 ('baby', 417),
 ('can', 395),
 ('when', 389),
 ('It', 388),
 ('one', 388),
 ('very', 378),
 ('be', 373),
 ('at', 365),
 ('just', 337),
 ('up', 337),
 ('or', 336),
 ('has', 336),
 ('her', 332),
 ('they', 332),
 ('our', 331),
 ('like', 327),
 ('use', 320),
 ('had', 297),
 ('these', 296),
 ('would', 294),
 ('We', 287),
 ('them', 286),
 ('little', 286),
 ('he', 284),
 ('This', 282),
 ('out', 270),
 ('great', 268),
 ('all', 263),
 ('easy', 261),
 ('she', 260),
 ('get', 254),
 ('if', 248),
 ('from', 241),
 ('really', 240),
 ("it's", 235),
 ('more', 234),
 ('because', 232),
 ('love', 225),
 ('only', 214),
 ('My

## 부정 리뷰 단어 빈도수 저장

In [277]:
# 각 리뷰를 word 별로 split한다
temp = []
temp.extend([sentence.split(' ') for sentence in neg_list])
# split된 각 리뷰를 flatten하게 만든다
neg_words = []
for sublist in temp:
    for item in sublist:
        neg_words.append(item)
# flatten 한 words 배열을 Counter 객체에 전달하여 단어별 빈도수를 저장한다
neg_counter = Counter(neg_words)
sorted(neg_counter.items(), reverse=True, key=lambda el: el[1])

[('the', 5251),
 ('to', 2930),
 ('I', 2751),
 ('and', 2690),
 ('a', 2452),
 ('it', 2252),
 ('', 1849),
 ('is', 1720),
 ('of', 1416),
 ('for', 1287),
 ('that', 1120),
 ('in', 1108),
 ('this', 1080),
 ('but', 926),
 ('on', 889),
 ('my', 873),
 ('not', 815),
 ('was', 789),
 ('have', 787),
 ('with', 763),
 ('you', 617),
 ('are', 564),
 ('so', 561),
 ('The', 538),
 ('be', 528),
 ('as', 503),
 ('just', 437),
 ('would', 421),
 ('at', 411),
 ('one', 399),
 ('when', 384),
 ('like', 375),
 ('baby', 373),
 ('It', 359),
 ('they', 357),
 ('out', 344),
 ('if', 341),
 ('up', 338),
 ('or', 337),
 ('we', 333),
 ('use', 327),
 ('get', 322),
 ('had', 300),
 ('very', 296),
 ('from', 292),
 ('can', 279),
 ('really', 276),
 ('more', 264),
 ('because', 263),
 ('these', 262),
 ('them', 258),
 ("it's", 254),
 ('will', 252),
 ('has', 251),
 ('all', 245),
 ('our', 231),
 ('it.', 227),
 ('only', 222),
 ('This', 221),
 ('seat', 218),
 ('We', 215),
 ('your', 210),
 ('an', 209),
 ('little', 208),
 ('too', 201),
 ('t

In [278]:
print(pos_counter['no'])
print(neg_counter['no'])

136
167


# 베이즈 정리
베이즈 정리를 적용하여 `This crib was amazing` 이란 리뷰의 긍정/부정을 분류해봅시다. 주어진 리뷰가 긍정일 확률을 계산하는 수식은 다음과 같습니다.

$$P(positive|review) = \frac{ {P(review|positive)} \cdot P(positive) }{P(review)}$$


제일 먼저 계산할 것은 $P(positive)$ 입니다. 위에서 우리는 1:1의 비율로 부정/긍정 리뷰를 추출하였으므로 해당 확률은 `0.5`가 됩니다.

In [279]:
percent_pos, percent_neg = 0.5, 0.5

다음으로 계산할 것은 `P(review | positive)` 입니다. 이 값은 주어진 리뷰가 긍정적일 때, `This`, `crib`, `was`, 그리고  `amazing` 총 4가지 단어들만이 존재할 확률을 의미합니다. 이 값을 계산하기 위해선 각 단어들이 모두 독립적이라는 가정이 필요합니다. 이는 하나의 단어가 다음 단어가 나타나는 사건에 대해 영향을 미치지 않는다는 것입니다. 이러한 가정을 하게 되면 각 단어들이 나타나는 사건은 독립 사건이므로 확률을 다음과 같이 계산할 수 있습니다.

$$P("This~crib~was~amazing" | positive) = P("This" | positive) \cdot P("crib" | positive) \cdot P("was" | positive) \cdot P("amazing" | positive)$$



In [280]:
review = "This crib was amazing"
total_pos = sum(pos_counter.values())	# 긍정 리뷰에 포함된 전체 단어 수
total_neg = sum(neg_counter.values())	# 부정 리뷰에 포함된 전체 단어 수
review_appear_in_pos_prob = 1
review_appear_in_neg_prob = 1

for word in review.split(' '):
    review_appear_in_pos_prob *= (pos_counter[word] + 1) / (total_pos + len(pos_counter))
    review_appear_in_neg_prob *= (neg_counter[word] + 1) / (total_neg + len(neg_counter))
    
print(review_appear_in_pos_prob)
print(review_appear_in_neg_prob)

5.895825521028269e-13
1.0430004646192789e-13


이제 남은 것은 `P(review)` 뿐입니다. 이 값은 `This`, `crib`, `was`, 그리고  `amazing` 총 4개의 단어들만 리뷰에서 나타날 확률을 의미합니다. 이는 위에서 계산한 `P(review | positive)` 와 매우 유사하지만 주어진 리뷰가 긍정적이라고 가정하지 않는다는 차이점을 갖습니다. 

그런데 `P(review)` 를 계산하기 전에 잠깐 생각해봅시다. 우리의 궁극적인 목표는 임의의 리뷰가 주어졌을 때 해당 리뷰가 긍정적인지 부정적인지를 알고싶다는 것입니다. 즉, `P(positive | review)` 와 `P(negative | review)` 둘 중에 어느 값이 더 큰지 확인하고 싶다는 것입니다. 두 확률을 계산하는 수식을 풀어쓰면 다음과 같습니다.

$$P(positive | review) = \frac{P(review | positive) \cdot P(positive)}{P(review)}$$

$$P(negative | review) = \frac{P(review | negative) \cdot P(negative)}{P(review)}$$

비교하고자 하는 두 값의 분모가 `P(review)` 로 동일합니다. 즉, 똑같은 값 `P(review)` 를 계산해서 나누어줄 필요 없이 그냥 무시해버리면 되는 것입니다. 그러므로 `P(review)` 를 계산해서 나누어주는 부분을 생략할 수 있으며, 지금까지 계산 정보를 바탕으로 주어진 리뷰가 긍정적인지 부정적인지를 판단할 수 있게 됩니다.

In [281]:
final_pos = review_appear_in_pos_prob * percent_pos
final_neg = review_appear_in_neg_prob * percent_neg
print(final_pos)
print(final_neg)

2.9479127605141343e-13
5.2150023230963945e-14


# Formatting Data for scikit-learn
지금까지 순수하게 수식을 활용하여 나이브 베이즈 분류 모델을 구현하였습니다. 그러나 Scikit-learn 라이브러리를 활용하면 코드의 양을 훨씬 줄일 수 있습니다. Scikit-learn의 나이브 베이즈 모델을 활용하기 위해선 먼저 데이터를 scikit-learn이 활용할 수 있는 형태로 transform해주어야 합니다. 이를 위해 scikit-learn에서 제공하는 `CountVectorizer` 객체를 사용할 것입니다.

먼저 `CountVectorizer` 객체를 생성하고 training set으로 vocabulary를 학습시킵니다. 다른 모델들과 마찬가지로 `.fit()` 메서드를 활용하여 학습시킵니다.

In [282]:
vectorizer = CountVectorizer()
vectorizer.fit(neg_list + pos_list)
print(vectorizer.vocabulary_)

{'registered': 6517, 'for': 3346, 'this': 8218, 'swing': 7984, 'my': 5229, 'son': 7463, 'as': 624, 'it': 4347, 'was': 8919, 'one': 5483, 'of': 5454, 'the': 8156, 'few': 3165, 'non': 5351, 'feminine': 3159, 'swings': 7986, 'could': 1989, 'find': 3193, 'thought': 8225, 'very': 8801, 'cute': 2159, 'gender': 3546, 'neutral': 5307, 'have': 3854, 'been': 902, 'extremely': 3048, 'disappointed': 2450, 'motor': 5180, 'on': 5480, 'is': 4337, 'joke': 4403, 'and': 503, 'what': 9003, 'worse': 9139, 'we': 8956, 'exchanged': 2953, 'had': 3774, 'same': 6853, 'problems': 6170, 'so': 7410, 'know': 4499, 'isn': 4342, 'just': 4430, 'bad': 775, 'loud': 4786, 'enough': 2853, 'that': 8152, 'broken': 1211, 'when': 9010, 'first': 3212, 'tried': 8465, 'out': 5557, 'but': 1311, 'barely': 813, 'has': 3842, 'power': 6065, 'to': 8315, '12': 14, 'pound': 6059, 'literally': 4716, 'if': 4078, 'leave': 4609, 'its': 4354, 'own': 5634, 'inch': 4127, 'either': 2773, 'way': 8953, 'with': 9094, 'in': 4120, 'he': 3863, 'only

Vectorizer를 학습시킨 뒤 `.transform()` 메서드를 호출할 수 있습니다. `.transform()` 메서드는 문자열의 배열을 받아서 학습된 단어들의 갯수로 변환합니다. 

In [283]:
training_counts = vectorizer.transform(neg_list + pos_list)
print(training_counts.shape)
print(training_counts.toarray())

(2000, 9239)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Using scikit-learn

In [284]:
review_counts = vectorizer.transform([review])

classifier = MultinomialNB()
training_labels = [0] * 1000 + [1] * 1000

classifier.fit(training_counts, training_labels)
print(classifier.predict(review_counts))
print(classifier.predict_proba(review_counts))

[1]
[[0.23763542 0.76236458]]
